# Imports for Face Mask Detection System
**(Essential libraries for video processing, image manipulation, and deep learning model inference)**

**Breakdown of Imports:**
* cv2 (OpenCV) - Real-time video capture and image processing

* imutils - Video stream utilities and image resizing

* numpy - Array operations for image data

* VideoStream - Webcam/video input handling

* load_model - Loads pre-trained mask detection model

* img_to_array + preprocess_input - Prepares frames for neural network input

In [1]:
import cv2 #
import imutils 
import numpy as np 
from imutils.video import VideoStream 
from tensorflow.keras.models import load_model 
from tensorflow.keras.preprocessing.image import img_to_array 
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input 

# Face Detection and Mask Prediction Function
*(Processes video frames to detect faces and classify mask/no-mask using deep learning)*

**Key Operations:**
1. Face Detection

* Uses OpenCV's DNN module with a pre-trained face detector (faceNet)

* Creates image blobs (224×224) and processes detections

2. ROI Processing

* Extracts face regions-of-interest (ROIs)

* Preprocesses faces (resize, BGR→RGB, normalization) for the mask classifier

3. Mask Prediction

* Runs batch inference using the mask classifier (maskNet)

* Returns face locations and mask predictions (probability scores)

Input: Video frame + face detection model + mask classifier model
Output: Tuple of (face bounding boxes, mask predictions)

In [4]:
def detect_and_predict_mask(frame, faceNet, maskNet):

	(h, w) = frame.shape[:2]
	blob = cv2.dnn.blobFromImage(frame, 1.0, (224, 224),
		(104.0, 177.0, 123.0))

	faceNet.setInput(blob)
	detections = faceNet.forward()
	print(detections.shape)

	faces = []
	locs = []
	preds = []

	for i in range(0, detections.shape[2]):
		confidence = detections[0, 0, i, 2]

		if confidence > 0.5:
			box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
			(startX, startY, endX, endY) = box.astype("int")
			(startX, startY) = (max(0, startX), max(0, startY))
			(endX, endY) = (min(w - 1, endX), min(h - 1, endY))
			face = frame[startY:endY, startX:endX]
			face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
			face = cv2.resize(face, (224, 224))
			face = img_to_array(face)
			face = preprocess_input(face)
			faces.append(face)
			locs.append((startX, startY, endX, endY))

	if len(faces) > 0:
		faces = np.array(faces, dtype="float32")
		preds = maskNet.predict(faces, batch_size=32)

	return (locs, preds)

# Load Face Detection and Mask Classification Models
**This cell initializes two deep learning models:**

1. Face Detection Model (Caffe-based SSD)

* Architecture: deploy.prototxt

* Pretrained weights: res10_300x300_ssd_iter_140000.caffemodel

* Loaded via OpenCV's DNN module (cv2.dnn.readNet)

2. Mask Classifier (Keras model)

* Pretrained model: mask_detector.model.h5

* Loaded using TensorFlow/Keras (load_model)

Paths point to local model files in the project directory.

In [5]:
prototxtPath = r"C:\AI projects\Face_Mask_Detection-main\face detector files\deploy.prototxt"
weightsPath = r"C:\AI projects\Face_Mask_Detection-main\face detector files\res10_300x300_ssd_iter_140000.caffemodel"
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

maskNet = load_model("mask_detector.model.h5")

# Real-Time Face Mask Detection Loop
**This cell implements the core live detection pipeline:**

1. Video Stream Initialization

* Starts webcam feed (src=0) using VideoStream

* Resizes frames to 400px width for consistent processing

2. Mask Detection & Visualization

* Detects faces and classifies mask status via detect_and_predict_mask()

* Annotates frames with:

* Bounding boxes (green=Mask, red=No Mask)

* Confidence percentages (e.g., "Mask: 98.50%")

3. Interactive Control

* Press ESC (27) or 'q' to quit

* Releases resources (destroyAllWindows(), stop()) on exit

**Note: Uses OpenCV for real-time display and imutils for optimized video handling.**

In [6]:
vs = VideoStream(src=0).start()

while True:

	frame = vs.read()
	frame = imutils.resize(frame, width=400)
	(locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)
	for (box, pred) in zip(locs, preds):
		(startX, startY, endX, endY) = box
		(mask, withoutMask) = pred

		label = "Mask" if mask > withoutMask else "No Mask"
		color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

		label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

		cv2.putText(frame, label, (startX, startY - 10),
		cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
		cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

	cv2.imshow("Frame", frame)
	key = cv2.waitKey(1)

	if key == 27 or key == ord('q'):
		break
cv2.destroyAllWindows()
vs.stop()

(1, 1, 200, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
(1, 1, 200, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
(1, 1, 200, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
(1, 1, 200, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
(1, 1, 200, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
(1, 1, 200, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
(1, 1, 200, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
(1, 1, 200, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
(1, 1, 200, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
(1, 1, 200, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
(1, 1, 200, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
(1, 1, 200, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
(1, 1, 200, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
(1, 1, 200, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
(1, 1, 200, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
(1, 1, 200, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
(1, 1, 200, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
(1, 1, 200, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
(1, 1, 200, 7)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/